In [ ]:
import os
import json
from huggingface_hub import InferenceClient
from dotenv import load_dotenv # dotenv access

load_dotenv() # load .env file
KEY = os.getenv("HF_API_KEY")

In [ ]:
# Defining the models we might use
model_id = "Qwen/Qwen2.5-72B-Instruct"
alt_model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

In [ ]:
# Client intitialization
client = InferenceClient(api_key=KEY, model=model_id)

================================================================================================================================
# Test Code
### Understand the structure of LLM query passing.

In [ ]:
messages = [
	{"role": "user", "content": "Who is the prime minister of India?"}
]
resp = client.chat.completions.create(messages, max_tokens=128)

In [ ]:
print(resp.choices[0].message.content)

================================================================================================================================
# Quiz generation
### Generates quiz MCQs based on the input provided.

In [ ]:
response_json = {
    "1" : {
        "mcq" : "multiple choice question",
        "options" : {
            "A" : "choice here",
            "B" : "choice here",
            "C" : "choice here",
            "D" : "choice here"
        },
        "correct" : "correct answer",
    },
    "2" : {
        "mcq" : "multiple choice question",
        "options" : {
            "A" : "choice here",
            "B" : "choice here",
            "C" : "choice here",
            "D" : "choice here"
        },
        "correct" : "correct answer",
    },
    "3" : {
        "mcq" : "multiple choice question",
        "options" : {
            "A" : "choice here",
            "B" : "choice here",
            "C" : "choice here",
            "D" : "choice here"
        },
        "correct" : "correct answer",
    }
}

In [ ]:
template_generation = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guid. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [ ]:
input_text="When discussing quantum computers, it is important to understand that at the smallest scales, the universe behaves very differently from what we are used to in our day-to-day lives. Compared to what we learned in grade-school physics, the behaviors of quantum objects are often bizarre and counterintuitive. Describing the behaviors of quantum particles presents a unique challenge. Most common-sense paradigms for the natural world lack the vocabulary to communicate the surprising behaviors of quantum particles. But quantum mechanics reveals how the universe really works. Quantum computers take advantage of quantum mechanics by replacing traditional binary bit circuits with quantum particles called quantum bits, or qubits. These particles behave differently from bits, exhibiting unique properties that can be described only with quantum mechanics. To understand quantum computing, it is important to understand four key quantum mechanics principlesSuperposition Entanglement Decoherence Interference Superposition A qubit itself isn't very useful. But it can place the quantum information it holds into a state of superposition, which represents a combination of all possible configurations of the qubit. Groups of qubits in superposition can create complex, multidimensional computational spaces. Complex problems can be represented in new ways in these spaces. When a quantum system is measured, its state collapses from a superposition of possibilities into a binary state, which can be registered like binary code as either a zero or a one. Entanglement Entanglement is the ability of qubits to correlate their state with other qubits. Entangled systems are so intrinsically linked that when quantum processors measure a single entangled qubit, they can immediately determine information about other qubits in the entangled system. Interference Interference is the engine of quantum computing. An environment of qubits placed into a state of collective superposition structures information in a way that looks like waves, with amplitudes associated with each outcome. These amplitudes become the probabilities of the outcomes of a measurement of the system. These waves can build on each other when many of them peak at a particular outcome or cancel each other out when peaks and troughs interact. Amplifying a probability or canceling out others are both forms of interference. Decoherence Decoherence is the process in which a system in a quantum state collapses into a nonquantum state. It can be intentionally triggered by measuring a quantum system or by other environmental factors (sometimes these factors trigger it unintentionally). Generally speaking, quantum computing requires avoiding and minimizing decoherence."
input_number=2
input_subject="Quantum Computing"
input_tone="Professional"
str_json=json.dumps(response_json)

In [ ]:
print(type(str_json), type(response_json))

In [ ]:
def generate_quiz(text, number, subject, tone, response_json):
    formatted_prompt = template_generation.format(
        text=text,
        number=number,
        subject=subject,
        tone=tone,
        response_json = str_json # ensures the prompt contains valid JSON string instead of Python dict
    )
    messages = [{"role": "user", "content": formatted_prompt}]
    output = client.chat_completion(
        messages=messages,
        model=model_id,
        max_tokens=1000
    )
    return output.choices[0].message.content

In [ ]:
primary_quiz = generate_quiz(
    text=input_text, 
    number=input_number,
    subject=input_subject, 
    tone=input_tone,
    response_json=str_json
)

print(primary_quiz)

# Generate and Review > independent
### Generates MCQ quiz questions with correct options and reviews the quiz questions and provides updated quiz questions if it doesn't match the complexity for student's subject.

In [ ]:
# Template for reviewing the code
template_review = """
You are an expert english grammarian and writer. Given a MCQ for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only \
use at max 50 words for complexity if the quiz is not at par with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student's ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz.
"""

In [ ]:
def review_quiz(quiz_text, subject):
    # We inject the output of the first function (quiz_text) into this template
    formatted_prompt = template_review.format(
        subject=subject,
        quiz=quiz_text
    )
    
    messages = [{"role": "user", "content": formatted_prompt}]
    output = client.chat_completion(
        messages=messages,
        model=model_id,
        max_tokens=1000
    )
    return output.choices[0].message.content

In [ ]:
# --- Reviewing & Refining Quiz ---
reviewed_quiz = review_quiz(
    quiz_text=primary_quiz, # Passing the result of Primary quiz
    subject=input_subject
)

In [ ]:
print("--- Primary Quiz ---")
print(primary_quiz)
print("\n\n--- Reviewed Quiz ---")
print(reviewed_quiz)

# Refined Chain Code

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
llm_client = HuggingFaceEndpoint(
    repo_id=model_id,
    max_new_tokens=1024,
    temperature=0.7,
    huggingfacehub_api_token=KEY
)

In [ ]:
chat_model = ChatHuggingFace(llm=llm_client)

In [ ]:
gen_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template_generation
)

In [ ]:
review_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=template_review
)

In [ ]:
# Chain 1: Generator
# This chain takes the raw inputs and outputs the Quiz JSON string
quiz_gen_chain = gen_prompt | chat_model | StrOutputParser()
type(quiz_gen_chain)

In [ ]:
# Chain 2: Reviewer
# This chain takes the quiz from Chain 1 and the subject, and outputs the review
review_chain = review_prompt | chat_model | StrOutputParser()
type(review_chain)

In [ ]:
overall_chain = (
    RunnablePassthrough.assign(quiz=quiz_gen_chain)  # Runs gen_chain, stores result in 'quiz' key
    | RunnablePassthrough.assign(review=review_chain) # Runs review_chain using 'quiz' from prev step
)

In [ ]:
result = overall_chain.invoke({
    "text": input_text,
    "number": input_number,
    "subject": input_subject,
    "tone": input_tone,
    "response_json": str_json
})

In [ ]:
print(f"--- Generated Quiz ---\n{result['quiz']}")

In [ ]:
print(f"\n--- Expert Review ---\n{result['review']}")